In [ ]:
!pip install timm

In [ ]:
import numpy as np 
import pandas as pd 
import os
import cv2
import torch.nn.init as init
import torch
import torch.nn as nn
from torch.nn.modules.loss import _WeightedLoss
from PIL import Image, ImageFilter
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Dataset
from torchvision import transforms
from torch.optim import Adam, SGD, RMSprop
import time
from torch.autograd import Variable
import torch.functional as F
from tqdm import tqdm
from sklearn import metrics
import urllib
import pickle
import cv2
import torch.nn.functional as F
from torchvision import models
import seaborn as sns
import random
import timm
from sklearn.metrics import roc_auc_score
import sys
sys.path.append('../input/autoaug')
from auto_augment import AutoAugment, Cutout
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from catalyst.data.sampler import BalanceClassSampler

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True

In [ ]:
seed_everything(0)
num_classes = 2
bs = 32
lr = 1e-4
IMG_SIZE = 256

In [ ]:
train_path = '../input/melanoma-merged-external-data-512x512-jpeg/512x512-dataset-melanoma/512x512-dataset-melanoma/'
test_path = '../input/melanoma-merged-external-data-512x512-jpeg/512x512-test/512x512-test/'
train_csv = pd.read_csv('../input/melanoma-merged-external-data-512x512-jpeg/folds_13062020.csv')
test_csv = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sample = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')

In [ ]:
class MyDataset(Dataset):
    
    def __init__(self, dataframe, transform=None, test=False):
        self.df = dataframe
        self.transform = transform
        self.test = test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        label = self.df.target.values[idx]
        
        
        if self.test == False:
            p = self.df.image_id.values[idx]
            p_path = train_path + p + '.jpg'
        else:
            p = self.df.image_name.values[idx]
            p_path = test_path + p + '.jpg'
            
        image = cv2.imread(p_path, cv2.IMREAD_COLOR)
        image = image.astype(np.float32) / 255.0
#         image = transforms.ToPILImage()(image)
        
        if self.transform:
            sample = {'image': image}
            sample = self.transform(**sample)
            image = sample['image']
        
        return image, label
    
    def get_labels(self):
        return list(self.df.target.values)

In [ ]:
class AdvancedHairAugmentation:
    def __init__(self, hairs: int = 4, hairs_folder: str = ""):
        self.hairs = hairs
        self.hairs_folder = hairs_folder

    def __call__(self, img):
        n_hairs = random.randint(0, self.hairs)

        if not n_hairs:
            return img

        height, width, _ = img.shape  # target image width and height
        hair_images = [im for im in os.listdir(self.hairs_folder) if 'png' in im]

        for _ in range(n_hairs):
            hair = cv2.imread(os.path.join(self.hairs_folder, random.choice(hair_images)))
            hair = cv2.flip(hair, random.choice([-1, 0, 1]))
            hair = cv2.rotate(hair, random.choice([0, 1, 2]))

            h_height, h_width, _ = hair.shape  # hair image width and height
            roi_ho = random.randint(0, img.shape[0] - hair.shape[0])
            roi_wo = random.randint(0, img.shape[1] - hair.shape[1])
            roi = img[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            img2gray = cv2.cvtColor(hair, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
            hair_fg = cv2.bitwise_and(hair, hair, mask=mask)

            dst = cv2.add(img_bg, hair_fg)
            img[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst

        return img

In [ ]:
train_transform = A.Compose([
#             AdvancedHairAugmentation(hairs_folder='/kaggle/input/melanoma-hairs/'),
            A.RandomSizedCrop(min_max_height=(200, 200), height=256, width=256, p=0.5),
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=256, width=256, p=1),
            A.Cutout(num_holes=4, max_h_size=16, max_w_size=16, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),                  
            ], p=1.0)

test_transform = A.Compose([
            A.Resize(height=256, width=256, p=1.0),
            ToTensorV2(p=1.0),
            ], p=1.0)


testset      = MyDataset(sample, transform=test_transform, test=True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size=bs, shuffle=False, num_workers=4)

In [ ]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix(data, targets, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]
    
    lam = np.random.beta(alpha, alpha)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    data[:, :, bbx1:bbx2, bby1:bby2] = data[indices, :, bbx1:bbx2, bby1:bby2]
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))

    targets = [targets, shuffled_targets, lam]
    return data, targets

# loss 变化
def cutmix_criterion(preds, targets):
    targets1, targets2, lam = targets[0], targets[1], targets[2]
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(preds, targets1) + (1 - lam) * criterion(preds, targets2)


def mixup(data, targets, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]
    
    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets1 = [targets, shuffled_targets, lam]

    return data, targets1

def mixup_criterion(preds, targets):
    targets1, targets2, lam = targets[0], targets[1], targets[2]
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(preds, targets1) + (1 - lam) * criterion(preds, targets2)



In [ ]:
class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.1):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth_one_hot(targets: torch.Tensor, n_classes: int, smoothing=0.1):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = torch.empty(size=(targets.size(0), n_classes),
                                  device=targets.device) \
                .fill_(smoothing / (n_classes - 1)) \
                .scatter_(1, targets.data.unsqueeze(1), 1. - smoothing)
        return targets

    def forward(self, inputs, targets):
        targets = SmoothCrossEntropyLoss._smooth_one_hot(targets, inputs.size(-1),
                                                         self.smoothing)
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
def margin_focal_binary_cross_entropy(logit, truth):
    weight_pos=2
    weight_neg=1
    gamma=2
    margin=0.2
    em = np.exp(margin)
    
    l1 = nn.CrossEntropyLoss()(logit, truth)
    
    logit = logit[:,1].view(-1)
    truth = truth.view(-1)
    log_pos = -F.logsigmoid( logit)
    log_neg = -F.logsigmoid(-logit)

    log_prob = truth*log_pos + (1-truth)*log_neg
    prob = torch.exp(-log_prob)
    margin = torch.log(em +(1-em)*prob)

    weight = truth*weight_pos + (1-truth)*weight_neg
    loss = margin + weight*(1 - prob) ** gamma * log_prob

    loss = loss.mean()
    
    loss - loss*0.5+l1*0.5
    return loss

In [ ]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def train_model(model, epoch):
    model.train() 
    
    losses = AverageMeter()
    avg_loss = 0.

    optimizer.zero_grad()
    
    tk = tqdm(train_loader, total=len(train_loader), position=0, leave=True)
    for idx, (imgs, labels) in enumerate(tk):
        imgs_train, labels_train = imgs.cuda(), labels.cuda().long()
        
        if np.random.rand()<0.5:
            with torch.no_grad():
                imgs_train, targets = mixup(imgs_train, labels_train, 0.2)

            output_train = model(imgs_train)
            loss = mixup_criterion(output_train, targets) 
        else: # elif np.random.rand()<0.75:
            with torch.no_grad():
                imgs_train, targets = cutmix(imgs_train, labels_train, 1.0)
            output_train = model(imgs_train)
            loss = mixup_criterion(output_train, targets) 
            
#         output_train = model(imgs_train)
#         loss = criterion(output_train, labels_train)
        loss.backward()

        optimizer.step() 
        optimizer.zero_grad() 
        
        avg_loss += loss.item() / len(train_loader)
        
        losses.update(loss.item(), imgs_train.size(0))

        tk.set_postfix(loss=losses.avg)
        
    return avg_loss


def test_model(model):    
    model.eval()
    
    losses = AverageMeter()
    avg_val_loss = 0.
    
    valid_preds, valid_targets = [], []
    
    with torch.no_grad():
        tk = tqdm(val_loader, total=len(val_loader), position=0, leave=True)
        for idx, (imgs, labels) in enumerate(tk):
            imgs_valid, labels_valid = imgs.cuda(), labels.cuda().long()
            output_valid = model(imgs_valid)
            
            loss = criterion(output_valid, labels_valid)
            
            avg_val_loss += loss.item() / len(val_loader)

            losses.update(loss.item(), imgs_valid.size(0))
            
            tk.set_postfix(loss=losses.avg)
            
            valid_preds.append(torch.softmax(output_valid,1)[:,1].detach().cpu().numpy())
            valid_targets.append(labels_valid.detach().cpu().numpy())
            
        valid_preds = np.concatenate(valid_preds)
        valid_targets = np.concatenate(valid_targets)
        auc =  roc_auc_score(valid_targets, valid_preds) 
            
    return avg_val_loss, auc

In [ ]:
for i in range(1):
    fold = i+1
    print('fold:', fold)

    train_df = train_csv[(train_csv['fold'] != i)]
    val_df   = train_csv[(train_csv['fold'] == i) & (train_csv['source'] == 'ISIC20')]
    train_df.reset_index(drop=True, inplace=True)
    val_df.reset_index(drop=True, inplace=True)

    trainset = MyDataset(train_df, transform=train_transform)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=bs, shuffle=True,
#                 sampler=BalanceClassSampler(labels=trainset.get_labels(), mode="downsampling"),
                num_workers=4)
   
    valset = MyDataset(val_df, transform=test_transform)
    val_loader = torch.utils.data.DataLoader(valset, batch_size=bs, shuffle=False, num_workers=4)

    model = timm.create_model('resnext50_32x4d', pretrained=True, num_classes=num_classes)
    model.cuda()

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
#     criterion = nn.CrossEntropyLoss()
    criterion = margin_focal_binary_cross_entropy
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=1,
        threshold=1e-4,
        mode="max"
    )
    
    best_auc = 0
    n_epochs = 25

    for epoch in range(n_epochs):
        avg_loss = train_model(model, epoch)
        avg_val_loss, auc = test_model(model)

        if auc > best_auc:
            best_auc = auc
            torch.save(model.state_dict(), str(fold) + 'weight.pt')
       
        print('current_val_auc:', auc, 'best_val_auc:', best_auc)
        
        scheduler.step(auc)

In [ ]:
print(best_auc)

In [ ]:
model1 = timm.create_model('resnext50_32x4d', pretrained=True, num_classes=num_classes)
model1.cuda()
model1.load_state_dict(torch.load("./1weight.pt"))

# model2 = timm.create_model('resnext50_32x4d', pretrained=True, num_classes=num_classes)
# model2.cuda()
# model2.load_state_dict(torch.load("./2weight.pt"))

# model3 = timm.create_model('resnext50_32x4d', pretrained=True, num_classes=num_classes)
# model3.cuda()
# model3.load_state_dict(torch.load("./3weight.pt"))

# model4 = timm.create_model('resnext50_32x4d', pretrained=True, num_classes=num_classes)
# model4.cuda()
# model4.load_state_dict(torch.load("./4weight.pt"))

# model5 = timm.create_model('resnext50_32x4d', pretrained=True, num_classes=num_classes)
# model5.cuda()
# model5.load_state_dict(torch.load("./5weight.pt"))

In [ ]:
model1.eval()
# model2.eval()
# model3.eval()

In [ ]:
# models = [model1, model2, model3]
# ensemble = []

In [ ]:
# for j in range(len(models)):
    test_pred = np.zeros((len(sample),))

    with torch.no_grad():
        for i, data in enumerate(tqdm(test_loader, position=0, leave=True)):
            images, _ = data
            images = images.cuda()

            pred = model1(images)
            
            pred = torch.softmax(pred,1).cpu().detach().numpy()[:,1]

            test_pred[i*bs: (i+1)*bs] = pred

    sample.target = test_pred
    sample.to_csv('submission1.csv',index=False)
    
#     ensemble.append(test_pred)

In [ ]:
# ensemble = np.array(ensemble).mean(0)

# sample.target = ensemble
# sample.to_csv('submission.csv',index=False)